In [379]:
pip install plotly
pip install nbformat

In [381]:
from otoole import convert_results
from otoole import convert
import os
import plotly.graph_objs as go
import pandas as pd
import scripts.utilities as utils

* Create the Necessary Directories

In [382]:
os.makedirs('results', exist_ok=True)
os.makedirs('results/cbc_results', exist_ok=True)
os.makedirs('results/GLPK_results', exist_ok=True)
os.makedirs('results/visuals', exist_ok=True)
os.makedirs('solution', exist_ok=True)
os.makedirs('RES', exist_ok=True)

* Collect the User Configurations

In [383]:
config=utils.load_config('config/model_config.yml')

In [384]:
data_version=config['current_run']
print(f"Running the Model for {data_version}")

Running the Model for C_5.1


* Define Directories and Local Files

In [385]:
otoole_config=f'config/otoole_config.yaml'
solution=f'solution/BCF_{data_version}.sol'
ip_datafile=f'data/BCF_{data_version}.txt'
model='model/osemosys_short3.txt'
LP_file=f'LP_file/BCF_{data_version}.lp'
RES_diagram=f'RES/RES.png'
# result_file=f'Selected_Results_{data_version}.csv'

# 1- Input Data Preparation

 ## 1.1 Prepare Raw Data

> in progress...

## 1.2 Create GNU Mathprog Datafile

> in progress...

In [386]:
#!otoole convert csv datafile $data_csvs $ip_datafile $otoole_config
#print(f"Input Datafile Created and Saved to Directory: '{ip_datafile}'")

In [387]:
!otoole convert csv datafile data_files/{data_version} data/ip_datafile config/otoole_config.yaml
print(f"Datafile written: {ip_datafile}")
# print(f"Input Datafile Directory: {ip_datafile}")

Datafile written: data/BCF_C_5.1.txt


## 1.3 Collect the GNU Mathprog Model File

In [388]:
print(f"Model Directory: {model}")

Model Directory: model/osemosys_short3_EL.txt


# 2- Prepare the Linear Problem 

> bash cmds

In [389]:
print(f"Writing the LP with GLPK Solver for \nDatafile: {ip_datafile}\nModel: {model}")

Writing the LP with GLPK Solver for 
Datafile: data/BCF_C_5.1.txt
Model: model/osemosys_short3_EL.txt


In [390]:
# Delete all CSV files in the directory
!rm results/GLPK_results/*.csv

# 3- Solve the Liner Problem

## 3.1 GLPK

In [391]:
!glpsol -m $model -d data/ip_datafile --wlp $LP_file
# !glpsol -m $model -d $ip_datafile -o $result_file --wlp $LP_file
print(f"\n{60*'-'}\nGLPK results generated and saved as 'results/GLPK_results' ")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m model/osemosys_short3_EL.txt -d data/ip_datafile --wlp LP_file/BCF_C_5.1.lp
Reading model section from model/osemosys_short3_EL.txt...
model/osemosys_short3_EL.txt:768: warning: final NL missing before end of file
768 lines were read
Reading data section from data/ip_datafile...
3165 lines were read
Generating cost...
Generating CAa4_Constraint_Capacity...
Generating CAa5_TotalNewCapacity...
Generating CAb1_PlannedMaintenance...
Generating EBa10_EnergyBalanceEachTS4...
Generating EBa11_EnergyBalanceEachTS5...
Generating EBb4_EnergyBalanceEachYear4...
Generating S1_StorageLevelYearStart...
Generating S2_StorageLevelTSStart...
Generating SC8_StorageRefilling...
Generating SC1_LowerLimit...
Generating SC2_UpperLimit...
Generating SC7_StorageMaxUpperLimit...
Generating SI6_SalvageValueStorageAtEndOfPeriod1...
Generating SI7_SalvageValueStorageAtEndOfPeriod2...
Generating SI8_SalvageValueStorageAtEndOfPeriod3...


## 3.2 CBC

> Test purposes, not in use right now 

In [392]:
print(f"Solving the LP with CBC Solver\n"
      f"Datafile: {ip_datafile}\n"
      f"LP file:{LP_file}\n"
      f"Solution File:{solution}")
# !cbc $LP_file solve -solu $solution

Solving the LP with CBC Solver
Datafile: data/BCF_C_5.1.txt
LP file:LP_file/BCF_C_5.1.lp
Solution File:solution/BCF_C_5.1.sol


# 4- Extract the Results

## 4.1- Create CSV files from the Solution

In [393]:
print("Extracting the Results in CSV files")

Extracting the Results in CSV files


In [394]:
# convert_results(otoole_config, 'cbc', 'csv', solution, 'results', 'datafile', ip_datafile,False,model)
# print(f"Results Generated Successfully")

# 5- Visualize

## 5.1 RES

In [395]:
!otoole viz res datafile data/ip_datafile RES/RES.png config/otoole_config.yaml

## 5.2 Load Result File

> CAUTION: The scripts are tailored to fit for results assuming 1 region

In [396]:
# production_by_technology=utils.load_result_file('results/GLPK_results/ProductionByTechnology.csv')
production_by_technology=pd.read_csv( 'results/GLPK_results/ProductionByTechnology.csv', header=[2])

# use_by_technology=utils.load_result_file('results/GLPK_results/UseByTechnology.csv')
use_by_technology=pd.read_csv( 'results/GLPK_results/UseByTechnology.csv', header=[2])

# demand=utils.load_result_file('results/GLPK_results/Demand.csv')
demand=pd.read_csv( 'results/GLPK_results/Demand.csv', header=[2])

# Read the data from the CSV file into a pandas DataFrame
NewCapacity = pd.read_csv( 'results/GLPK_results/NewCapacity.csv', header=[1])

## 5.3 Production by Technology (Energy Units)

In [397]:
def generate_interactive_plots_gen(gen_df, data_version, plot_name, plot_save_to):
    df = gen_df

    # Get unique values of technology
    technologies = df['Technology'].unique()

    # Create a figure
    fig = go.Figure()

    # Create a scatter plot for each technology and add it to the figure
    for tech in technologies:
        # Filter the dataframe for the current technology
        filtered_df = df[df['Technology'] == tech]
        # Add trace for each technology
        fig.add_trace(go.Scatter(x=filtered_df['Timeslice'], y=filtered_df['2023'], mode='lines+markers', name=tech))

    # Update layout
    fig.update_layout(
        title=plot_name,
        xaxis_title='Timeslice',
        yaxis_title='Energy (MWh)',
        legend_title='Technology'
    )

    # Save the plot
    os.makedirs(f'results/visuals/data_version_{data_version}', exist_ok=True)
    plot_file_path = f'{plot_save_to}/data_version_{data_version}/{plot_name}-{data_version}.html'
    fig.write_html(plot_file_path)

    # Show a confirmation message
    print(f'Energy Plot for all technologies generated successfully and saved to {plot_file_path}')
    fig.show()

    return

In [398]:
def generate_interactive_plots_dem(dataframe, data_version,plot_name,plot_save_to):
    df = dataframe

    # Get unique values of technology
    Fuels = df['Fuel'].unique()

    # Create a figure
    fig = go.Figure()

    # Create a scatter plot for each technology and add it to the figure
    for Fuel in Fuels:
        # Filter the dataframe for the current technology
        filtered_df = df[df['Fuel'] == Fuel]

        # Add trace for each technology
        fig.add_trace(go.Scatter(x=filtered_df['Timeslice'], y=filtered_df['2023'], mode='lines+markers', name=Fuel))

    # Update layout
    fig.update_layout(
        title=plot_name,
        xaxis_title='Timeslice',
        yaxis_title='Demand (MWh)',
        legend_title='Fuel'
    )

    # Save the plot
    os.makedirs(f'results/visuals/data_version_{data_version}', exist_ok=True)
    plot_file_path = f'{plot_save_to}/data_version_{data_version}/{plot_name}-{data_version}.html'
    fig.write_html(plot_file_path)

    # Show a confirmation message
    print(f'Demand Plot for all Fuels generated successfully and saved to {plot_file_path}')
    fig.show()

    return

## 5.4 Generate Plots

In [399]:
generate_interactive_plots_gen(production_by_technology,data_version,'Production by Technology','results/visuals')
generate_interactive_plots_gen(use_by_technology,data_version,'Use by Technology','results/visuals')
generate_interactive_plots_dem(demand,data_version,'Demand','results/visuals')

Energy Plot for all technologies generated successfully and saved to results/visuals/data_version_C_5.1/Production by Technology-C_5.1.html


Energy Plot for all technologies generated successfully and saved to results/visuals/data_version_C_5.1/Use by Technology-C_5.1.html


Demand Plot for all Fuels generated successfully and saved to results/visuals/data_version_C_5.1/Demand-C_5.1.html
